In [1]:
import torch as t

In [2]:
########################################################所有神经网络的核心包autograd 它为张量上的所有操作提供了自动求导机制。

###torch.Tensor 是这个包的核心类。如果设置它的属性 .requires_grad 为 True，那么它将会追踪对于该张量的所有操作。 
###当完成计算后可以通过调用根节点 .backward()，来自动计算所有的梯度。
###这个张量的所有梯度将会自动累加到.grad属性.

###要阻止一个张量被跟踪历史，可以调用 .detach() 方法将其与计算历史分离，并阻止它未来的计算记录被跟踪。或者包裹在with t.no_grad():代码块中

###还有一个类对于autograd的实现非常重要：Function。
###Tensor 和 Function 互相连接生成了一个无圈图(acyclic graph)，它编码了完整的计算历史。
###每个张量都有一个 .grad_fn 属性，该属性引用了创建 Tensor 自身的Function(除非这个张量是用户手动创建的，即这个张量的 grad_fn 是 None )。


###如果需要计算导数，可以在 Tensor 上调用 .backward()。
###如果 Tensor 是一个标量(即它包含一个元素的数据），则不需要为 backward() 指定任何参数，
###但是如果它有更多的元素，则需要指定一个 gradient 参数，该参数是形状匹配的张量。


In [3]:
##创建张量并追踪计算历史
x=t.ones(2,2,requires_grad=True)
print(x,x.grad_fn)#叶子节点grad_fn为none 是用户创造的 x.is_leaf=True
y=x+2
print(y,y.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True) None
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>) <AddBackward0 object at 0x000002286F67AF08>


In [4]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [5]:
###梯度的反向传播

###因为 out 是一个标量，因此 out.backward() 和 out.backward(torch.tensor(1.)) 等价。
out.backward()
print(x.grad)
###图片手动推导

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [6]:
#注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，
#梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()#   in-place操作清零
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


In [7]:
###为什么在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor? 
###简单来说就是为了避免向量（甚至更高维张量）对张量求导，而转换成标量对张量求导。
###举个例子，假设形状为 m x n 的矩阵 X 经过运算得到了 p x q 的矩阵 Y，Y 又经过运算得到了 s x t 的矩阵 Z。
###那么按照前面讲的规则，dZ/dY 应该是一个 s x t x p x q 四维张量，dY/dX 是一个 p x q x m x n的四维张量。

###问题来了，怎样反向传播？怎样将两个四维张量相乘？？？这要怎么乘？？？
###就算能解决两个四维张量怎么乘的问题，四维和三维的张量又怎么乘？导数的导数又怎么求，这一连串的问题，感觉要疯掉…… 

###为了避免这个问题，我们不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。
###所以必要时我们要把张量通过将所有张量的元素加权求和的方式转换为标量，
###举个例子，假设y由自变量x计算而来，w是和y同形的张量，则y.backward(w)的含义是：先计算l = torch.sum(y * w)，
###则l是个标量，然后求l对自变量x的导数。
x = t.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
print("y=",y)
z = y.view(2, 2)
print("z=",z)

y= tensor([2., 4., 6., 8.], grad_fn=<MulBackward0>)
z= tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [8]:
##################现在 z 不是一个标量，所以在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量。
v = t.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=t.float)
z.backward(v)
print(v.grad)
print(x.grad)
##图片手动推导

None
tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [9]:
###中断梯度追踪的例子

###为了防止跟踪历史记录(和使用内存），可以将代码块包装在 with torch.no_grad(): 中。
###在评估模型时特别有用，因为模型可能具有 requires_grad = True 的可训练的参数，但是我们不需要在此过程中对他们进行梯度计算。

x = t.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with t.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [10]:
#反向传播y3
y3.backward()
print(x.grad)

tensor(2.)


In [11]:
##如果我们想要修改tensor的数值，但是又不希望被autograd记录（即不会影响反向传播），那么我么可以对tensor.data进行操作。
x = t.ones(1,requires_grad=True)
print(x)
print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.], requires_grad=True)
tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])


In [12]:
###如果y= w * x， 计算y对w的梯度时需要用到x的值，这些数值会在前向传播过程中保存为buffer值，在梯度计算完成后释放，
##所以在backward()时要保留计算图
###反向传播过程中非叶子节点的导数计算完成即被清空，要查看可以用t.autograd.grad(z,y)查看 #z为根节点 y为中间结点